### Data_Loader Testing

In [1]:
import data_loader
import torch
from torch.utils.data import DataLoader
from pathlib import Path
from torchvision.transforms import v2 as transforms


In [2]:
test_dir = Path("mbrimberry_files\Submissions\\test")
train_dir = Path("mbrimberry_files\Submissions\\train")

In [ ]:
trans = transforms.Compose(
    [
        transforms.ToImage(),
        transforms.Resize(size=(1056,816))
        
    ])
train_data = data_loader.DeskTopDataset(targ_dir=train_dir,transform=trans)
test_data = data_loader.DeskTopDataset(targ_dir=test_dir,transform=trans)


In [ ]:
import matplotlib.pyplot as plt
image, targets = train_data[2]
print(f"{image.shape}")
image_permute = image.permute(1,2,0)
print(f"{image_permute.shape}")
fig, ax = plt.subplots(1)
ax.imshow(image_permute)

In [ ]:
img, targets = train_data[0]
print(img)
print(targets)

In [ ]:
len(train_data)


In [ ]:
len(test_data)

In [ ]:
test_data.classes

In [10]:
BATCH_SIZE= 32
NUM_WORKERS = 0

dataloader = DataLoader(dataset=train_data,
                        batch_size=BATCH_SIZE,
                        num_workers=NUM_WORKERS,
                        shuffle=False)

In [11]:
def collate_fn(batch):
    return{
        'img' : torch.stack([x['img'] for x in batch])
    }

In [ ]:
img, targets = next(iter(dataloader))
img[1]

In [ ]:
import torchvision
import torchvision.transforms.v2
from torchvision import utils
from torchvision.transforms.v2 import functional as F
viz = utils.draw_bounding_boxes(F.ToImage(img), boxes=bbox)
F.to_pil_image(viz).show()